#Weak-Strong Cluster問題

2015年にGoogleとNASAが共同でD-Waveマシンは既存マシンの１億倍高速という発表を行いました。その際に利用されたのが量子ビットのクラスタを作ってフリップさせるWeak-Strong Cluster問題です。今回は簡単なweak clusterとstrong clusterを作って見て計算を行います。

論文は下記を参照します。

What is the Computational Value of Finite Range Tunneling?

https://arxiv.org/abs/1512.02206

##背景
量子アニーリングは量子トンネル効果を利用した最適化マシンとして提案されていますが、ここでは、このトンネル効果がどのような計算上のメリットをもたらすかを検証しています。D-Wave2Xの量子アニーリングマシンは局所解同士を隔てるエネルギー障壁が高く、細い形状をしているような問題に対して有利で、Simulated Annealing(SA)にくらべても優位性があるといわれています。945量子ビットの場合で、SAにくらべて、およそ10^8倍高速（成功率99%）で、古典計算機でトンネル効果をシミュレートする量子モンテカルロ法（QMC）と比較しても同様に高速です。

##ハミルトニアンとSA、QA
今回検証を行う際にシミュレーテッドアニーリング（以下SA）と量子アニーリング（今回は量子モンテカルロ法をつかっているので、以下QMC）が使用されています。

ときたい問題は一緒で、ハミルトニアンと呼ばれるコスト関数を最小にするようにアルゴリズムが働き、その最小基底状態に至る過程がSAとQMCでは原理が異なります。

SAでは熱をシミュレートして、熱で基底状態の探索を行います。一方QMCでは熱の代わりに磁力を使って、量子トンネル効果を活用しながら探索を行います。

SAではあるコスト関数がある場合、グラフの起伏をきちんとなぞるようにエネルギー障壁（以下エナジーバリア）を超えて探索を行うためエネルギー関数のコストをあげて探索を行わないといけませんが、QMCの場合にはトンネル効果によりエナジーバリアを越えるために熱のコストを上げる必要がなく、確率的にトンネル効果で起伏の向こう側に到達できると考えられます。

これらのエナジーバリアをトンネル効果で越える条件もありますので、できるだけエナジーバリアの高さが高くて、障壁の厚みが薄い方が確率的に超えやすいので、SAで行う場合には、かなり条件が厳しく、QMCや量子アニーリングに有利な条件となります。この条件を人為的に問題を作ることで、SAに対して速度優位性を持たせようという検証です。

つまり、求めたいコスト関数に高くて薄いエナジーバリアがたくさんあるほどD-WaveマシンやQMCアルゴリズムが有利になると推測されます。

##Weak-Strong Cluster問題とは
Weak-Strong Clusterという２つの量子ビットのクラスターを繋げる問題です。D-Waveはキメラグラフという接続を使用しており、8量子ビットで１ユニットセルという単位です。このユニットセルを２つ用意した、16量子ビットの２つのクラスターを構成する問題を用意しています。

<img src='https://github.com/Blueqat/Wildqat/blob/master/examples_ja/img/023_1.png?raw=1'>


全ての量子ビットはキメラグラフで接続されており、ferromagneticカップリングで結合されています。値が同じになるような結合です。一方局所磁場と呼ばれる量子ビットが-1か+1になりやすいように設定されたパラメータが工夫されています。右側のクラスターはすべてh2=−1という値が設定されている一方で、左側のクラスターの量子ビットにはh1=0.44というように設定されています。これにより、計算過程において、左側の量子ビットがまとめて8個同時にフリップして右側のクラスターと揃うという過程がおきます。局所磁場の値が、左がweak-clusterで右がstrong-clusterでweak-strong cluster問題です。

これをさらに巨大につなげることで大きなクラスタを作ることができています。クラスター16量子ビットの組みを複数用意し、strong-cluster同士を4量子ビットの結合でferro/anti-ferroをランダムで+1or-1でつなぐことで巨大なweak-strong clusterを作ったとのこと。

<img src='https://github.com/Blueqat/Wildqat/blob/master/examples_ja/img/023_2.png?raw=1'>
引用：https://arxiv.org/abs/1512.02206

D-Waveにはところどころ不良量子ビットもあるので、それを避けるようにクラスターを配置し、上記の巨大クラスター構築では、黒丸が-1のstrong cluster。グレーが0.44のweak cluster。青い接続がferroで赤い接続がanti-ferroとなっています。

##実験の結果
結果1億倍程度の速度差が生まれたとなっています。参考は下図。

<img src='https://github.com/Blueqat/Wildqat/blob/master/examples_ja/img/023_4.png?raw=1'>
引用：https://arxiv.org/abs/1512.02206


##部分的回路の実装
少し実際のアルゴリズムでやって見ます。実用的に2クラスタを解いて見ます。とりあえず16量子ビットのクラスタを今回は検証して見たいと思います。

まず面白いのは、量子ビット同士の結合がすべてferromagneticということです。設定する値は論文と符号が逆ですが、すべて-1を入れます。


上記量子ビットで今回の実験の肝は量子ビットの局所磁場を設定するところで、上記オレンジの右側のクラスターの量子ビットの局所磁場の設定をすべて+1に。上記水色の左側のクラスターの量子ビットの局所磁場の設定を全て-0.44にします。また、便宜的に１６量子ビットに通し番号をふりました。局所磁場はh0からh15まで、量子ビット間の相互作用強度はJijの表記をJ0,4のように量子ビットの番号で表現します。

<img src='https://github.com/Blueqat/Wildqat/blob/master/examples_ja/img/023_1.png?raw=1'>

まずは何も考えずにSAをかけて見たいと思います。

##キメラグラフの実装
キメラグラフでの結合係数の決定をします。今回はせいぜい16量子ビットなので、そのまま16*16のmatrixを作って実現して見ます。wildqatに下記のQUBOmatrixをいれることで計算を行うことができます。

<img src='https://github.com/Blueqat/Wildqat/blob/master/examples_ja/img/023_5.png?raw=1'>

オレンジの-1がユニットセル内の16の結合。クラスタが２つあるので、合計32のユニットセル内部の結合があります。次に赤の-0.44はクラスタ1の局所磁場。青の+1はクラスタ2の局所磁場。紫はクラスタ間の-1の結合を表しています。論文とwildqatはプラスマイナスの符号が逆になっています。

##実行して見る
こちらをwildqatに入力して実行してみます。

In [1]:
!pip install blueqat

     |████████████████████████████████| 51kB 1.7MB/s 


In [5]:
import blueqat.opt as wq
import numpy as np
a = wq.opt()
a.J = np.zeros((16,16))

for i in range(8):
  a.J[i][i] = -0.44

for i in range(8,16):
  a.J[i][i] = 1

for i in range(4,8):
  for j in range(0,4):
    a.J[j][i] = -1
    
for i in range(12,16):
  for j in range(8,12):
    a.J[j][i] = -1

a.J[4][12] = -1
a.J[5][13] = -1
a.J[6][14] = -1
a.J[7][15] = -1

a.sa()


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

すべて0になりました。時々実行すると左側だけすべて1となり、右側が0となる局所解にも落ちました。

In [6]:
a.sa()

[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

##参考にD-Wave実機での実行結果
また、D-Wave本体でもやってみました。パラメータは論文と一緒です。

<img src='https://github.com/Blueqat/Wildqat/blob/master/examples_ja/img/023_6.png?raw=1'>

成功率98.6%で基底状態です。論文とほぼ同じ。shotは1000回にして見ました。

この問題はぜひ興味ある人は実装が難しくないので小さな問題からチャレンジして、大きな問題にチャレンジして欲しいです。実用というよりも研究の要素がとても大きかったと思います。正直SAでは局所解から最適解への相転移は容易ではないと思います。その辺りがD-WaveやQMCなどの量子アルゴリズムの利点なのかなと思いました。すべては左側のクラスターの量子ビットの局所磁場h0=−0.44という数字がポイントになるので、この値を調整して見ても勉強になるかと思います。